In [1]:
import os
import glob
import json
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import string

In [2]:
def dict_to_paragraph(persona, residence):
    priorities = persona["Top Priorities"]
    party = random_party()
    try:
        pain_points = persona["Pain points"]
    except:
        pain_points = persona["Pain Points"]
    
    return (f"Your name is {persona['Name']}, you are a {persona['Age']} year old {persona['Ethnicity']} {persona['Gender']}.\n"
            f"You live in {residence} and your annual income is ${persona['Income']}.\n"
            f"Your marital status is {persona['Marital Status']}, and your highest level of education is {persona['Education Level']}.\n"
            f"You work as a {persona['Occupation']}. You are described as '{persona['Description']}'.\n"
            f"Your top priorities are {priorities}. You are facing some challenges, including {pain_points}.\n"
            f"Your political affiliation/party is {party}")

In [3]:
parent_path = "Groups/"
# Define the directories to go through
directories = ['Groups/Group_1', 'Groups/Group_2', 'Groups/Group_3', 'Groups/Group_4']

personas = []
for i in range(0,11):
    group_p = []
    # For each directory
    # for directory in directories:

    directory = f'Groups/Group_{i}'
    # Use a wildcard (*) to find all .txt files in the directory
    for file_name in glob.glob(directory + '/*.txt'):
        print(file_name)
        # Open each file in read mode to get the data
        file_data = GPT.open_file(file_name)
        try:
            for data in json.loads(file_data):
    #         for key, value in data.items():
    #             print(f"{key}: {value}")
                personas.append(dict_to_paragraph(data, 'Montana, United States'))
                group_p.append(dict_to_paragraph(data, 'Montana, United States'))
        except:
            continue
    print(f"Num Personas in Group {i}:", len(group_p))

Groups/Group_0/group_0_batch_13.txt
Groups/Group_0/group_0_batch_3.txt
Groups/Group_0/group_0_batch_2.txt
Groups/Group_0/group_0_batch_12.txt
Groups/Group_0/group_0_batch_38.txt
Groups/Group_0/group_0_batch_0.txt
Groups/Group_0/group_0_batch_10.txt
Groups/Group_0/group_0_batch_11.txt
Groups/Group_0/group_0_batch_1.txt
Groups/Group_0/group_0_batch_39.txt
Groups/Group_0/group_0_batch_5.txt
Groups/Group_0/group_0_batch_15.txt
Groups/Group_0/group_0_batch_29.txt
Groups/Group_0/group_0_batch_28.txt
Groups/Group_0/group_0_batch_14.txt
Groups/Group_0/group_0_batch_4.txt
Groups/Group_0/group_0_batch_16.txt
Groups/Group_0/group_0_batch_6.txt
Groups/Group_0/group_0_batch_7.txt
Groups/Group_0/group_0_batch_17.txt
Groups/Group_0/group_0_batch_58.txt
Groups/Group_0/group_0_batch_49.txt
Groups/Group_0/group_0_batch_48.txt
Groups/Group_0/group_0_batch_51.txt
Groups/Group_0/group_0_batch_45.txt
Groups/Group_0/group_0_batch_44.txt
Groups/Group_0/group_0_batch_50.txt
Groups/Group_0/group_0_batch_46.txt


In [4]:
# Open the JSON file in read mode and load its contents
file_path = "new_personas_343.json"
with open(file_path, "r") as json_file:
    personas = json.load(json_file)

In [5]:
print("Total Personas:", len(personas))

Total Personas: 343


In [6]:
personas[75]

{'Name': 'Jeremy Harris',
 'Age': '75',
 'Ethnicity': 'White',
 'Gender': 'Male',
 'Income': '2200',
 'Education Level': "Bachelor's Degree or Higher",
 'Political Affiliation': 'Right Leaning',
 'Geographic Location': 'Urban Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Occupation': 'Retired Business Executive.',
 'Media Consumption': 'Based on the demographic information provided, Jeremy Harris is likely to trust and consume media outlets that align with his right-leaning political affiliation and his age range. As an expert in market research, I would recommend the following media outlets that Jeremy Harris may trust and consume regularly:\n\n1. Fox News - Jeremy Harris is likely to consume this media outlet regularly as it aligns with his right-leaning political affiliation.\n\n2. The Wall Street Journal - As a retired business executive, Jeremy Harris may trust and consume this media outlet for its financial news and economic analysis.\n\n3. The Drudge Report - Th

In [7]:
responses = []
question = "Do you plan to vote in the 2022 midterm election?"

question = "At this point in time, do you approve or disapprove of President Joe Biden's performance in office?\nApprove\nDisapprove\nNeither Approve/Disapprove"

question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"

total_prompt_tokens = 0
total_completion_tokens = 0


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    try:
        resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.6, tokens = 300)
    except:
        return "",0
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses = list(executor.map(thread_func, range(len(personas))))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

2 {
    "Reasoning": "As a conservative, I am generally against government intervention and prefer limited government. While I understand the burden of student loans, I do not believe it is the government's responsibility to forgive them. I also believe in personal responsibility and that individuals should be held accountable for their financial decisions. Therefore, I disapprove of President Joe Biden's student loan forgiveness plan.",
    "Answer": "Disapprove"
}
5 {
    "Answer": "Approve",
    "Reasoning": "As a retired teacher, education has always been a top priority for me. I believe that everyone should have access to quality education and that student loan debt should not be a barrier to achieving one's goals. President Biden's student loan forgiveness plan aligns with my values and beliefs, and I approve of his efforts to alleviate the burden of student loan debt for millions of Americans. Additionally, as a liberal and member of the Democratic party, I generally support the

11 {
    "Reasoning": "As a right-leaning individual, I am typically not in favor of government programs that provide financial assistance or forgiveness. However, as someone with a higher education level and a higher income, I understand the burden that student loans can place on individuals and families. Therefore, I am open to the idea of student loan forgiveness, but I would need to see more details about President Biden's plan before making a final decision. Additionally, as someone who values personal responsibility and hard work, I would want to ensure that any forgiveness plan is targeted towards individuals who have demonstrated a commitment to paying off their loans and contributing to society.",
    "Answer": "Neither Approve/Disapprove"
}
16 {
    "Reasoning": "As a conservative with a high school education and having a job as a retail sales associate, I believe in personal responsibility and hard work. I do not agree with the idea of student loan forgiveness, as I believe 

27 {
    "Reasoning": "As a retired rancher living in rural Montana, I am not burdened by student loans since I have already completed my education. However, I believe that higher education should be accessible and affordable to all individuals, regardless of their financial situation. Therefore, I approve of President Joe Biden's student loan forgiveness plan as it will provide relief to many Americans who are struggling with student loan debt and help make higher education more accessible to future generations.",
    "Answer": "Approve"
}
26 {
    "Reasoning": [
        "As a marketing manager with a Bachelor's Degree or higher, I understand the burden of student loans and the impact it can have on one's financial situation.",
        "As a conservative with a moderate income, I believe in limited government intervention and personal responsibility.",
        "However, I also believe in the importance of education and the opportunities it can provide.",
        "Considering President

38 {
    "Reasoning": "As a right-leaning individual, I am generally in favor of policies that prioritize individual responsibility and self-sufficiency. While I do appreciate the idea of student loan forgiveness and the potential benefits it could have for individuals struggling with debt, I ultimately do not support President Biden's plan. I believe that individuals should be responsible for paying back the loans they have taken out, and that forgiveness programs incentivize irresponsible borrowing and spending. Additionally, I am concerned about the potential costs of such a program and the impact it could have on the economy as a whole. ",
    "Answer": "Disapprove"
}
40 {
    "Reasoning": "As a conservative, I generally prioritize smaller government and individual responsibility. While I understand the burden of student loans, I do not believe it is the government's responsibility to forgive them. Additionally, as a high school graduate, I did not pursue higher education and did n

55 {
    "Reasoning": "As a conservative, I generally believe in individual responsibility and limited government intervention in the economy. While I understand the burden of student loan debt, I do not believe it is the government's role to forgive it. Additionally, I am concerned about the potential economic impact of such a large-scale forgiveness program. Therefore, I would have to say that I disapprove of President Joe Biden's student loan forgiveness plan.",
    "Answer": "Disapprove"
}
53 {
    "Reasoning": "As a conservative high school graduate farmer living in rural Montana, I am likely to disapprove of President Joe Biden's student loan forgiveness plan. I believe in personal responsibility and that individuals should be accountable for their own financial decisions, including taking out student loans. I also believe in limited government and that the government should not be in the business of forgiving individual debts. Additionally, as someone who did not attend college,

...

291 {
    "Reasoning": "As a high school graduate working as a retail sales associate in rural Montana, I have student loan debt and I struggle to make my monthly payments. Therefore, I am in favor of President Biden's student loan forgiveness plan. It would help me and many others like me to alleviate the financial burden of student debt and allow us to focus on other important things like saving for a home or starting a family. Additionally, as a liberal, I support policies that aim to reduce income inequality and provide greater access to education and economic opportunities for all individuals. Forgiving student loans is one way to help achieve these goals. ",
    "Answer": "Approve"
}
287 {
    "Reasoning": "As an 84-year-old American Indian or Alaska Native female with a Bachelor's degree or higher and a retired teacher living in urban Montana with an income of 19600, I do not have any student loans to be forgiven. Therefore, I do not have a personal stake in this issue. However,

301 {
    "Reasoning": "As a high school graduate with no mention of attending college, the student loan forgiveness plan does not directly affect me. However, I do have an opinion on the matter. Given my right-leaning political affiliation, I am generally in favor of limited government intervention and believe that individuals should take responsibility for their own financial decisions. While I sympathize with those burdened by student loans, I do not believe it is the government's responsibility to forgive those loans. Additionally, as someone living in rural Montana with a moderate income, I am concerned about the potential cost of such a program and the impact it could have on the economy. Overall, I would have to say that I disapprove of President Biden's student loan forgiveness plan.",
    "Answer": "Disapprove"
}
299 {
    "Reasoning": ["As a high school graduate with an income of $36,000, I have student loan debt that is a significant burden on my finances. Therefore, I am in

315 {
    "Reasoning": "As a person with a high school education working as an administrative assistant, I do not have any student loans. However, I am a left-leaning individual who believes in policies that support economic justice and equality. President Joe Biden's student loan forgiveness plan aligns with my political beliefs and I appreciate his efforts to address the student loan debt crisis. I also believe that providing relief to those burdened by student loans can have a positive impact on the economy and society as a whole.",
    "Answer": "Approve"
}
313 {
    "Reasoning": "As a conservative rancher living in rural Montana, I am likely to disapprove of President Joe Biden's student loan forgiveness plan. I believe in personal responsibility and paying off one's debts, and I do not think it is fair for taxpayers to have to foot the bill for other people's student loans. Additionally, as someone who did not attend college and instead chose to pursue a career in ranching, I do 

322 {
    "Reasoning": "As an 87-year-old retired carpenter with a high school education and a moderate income, I am not directly impacted by President Joe Biden's student loan forgiveness plan. However, I believe that education is an important investment in our future, and I support policies that make it more accessible and affordable to everyone. While I understand that there are concerns about the cost of such a plan, I believe that the benefits of reducing the burden of student debt for millions of Americans outweigh the potential costs. Therefore, I approve of President Joe Biden's student loan forgiveness plan.",
    
    "Answer": "Approve"
}
324 {
    "Reasoning": ["As someone with a high school education and lower income, I did not attend college and do not have any student loans to forgive. However, I am aware of the struggles that many people face with student loan debt and believe that it can be a burden on individuals and families. As a conservative, I generally support sm

336 {
    "Reasoning": "As an 18-year-old high school graduate with a conservative political affiliation, I am likely to disapprove of President Joe Biden's student loan forgiveness plan. While I do believe that student loan debt is a significant issue that needs to be addressed, I do not think that forgiving all student loans is the right solution. I believe that individuals should be responsible for their own debts and that forgiving loans would be unfair to those who have already paid off their loans or chose not to take out loans. Additionally, I am concerned about the cost of such a plan and the impact it could have on the economy and future generations.", 
    "Answer": "Disapprove"
}
337 {
    "Reasoning": "As a software developer with a bachelor's degree or higher and an income of $66,500, I am fortunate to have a certain level of economic stability and do not currently have any outstanding student loans. However, as a liberal, I generally support policies that aim to make high

# Process Responses

In [8]:
def count_answers(strings):
    count_disapprove = 0
    count_approve = 0
    count_neither = 0

    for string in strings:
        if string == "Disapprove":
            count_disapprove += 1
        elif string == "Approve":
            count_approve += 1
        elif string == "Neither Approve/Disapprove":
            count_neither += 1

    return count_disapprove, count_approve, count_neither


In [10]:
answers = []
for resp in responses:
    try:
        data = json.loads(resp)
    except:
        print("failed")
        continue
    answers.append(data['Answer'])
    
disapprove_count, approve_count, neither_count = count_answers(answers)
print("Disapprove count:", disapprove_count/343)
print("Approve count:", approve_count/343)
print("Neither count:", neither_count/343)

Disapprove count: 0.37317784256559766
Approve count: 0.5422740524781341
Neither count: 0.08454810495626822


In [78]:
responses_2 = []
#question = "Do you plan to vote in the 2024 midterm election?\n Yes\n No\n Undecided\n Please note, you are allowed to say no and not voted"

question = "At this point in time, do you approve or disapprove of President Joe Biden's performance in office?\nApprove\nDisapprove\nNeither Approve/Disapprove"

#question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"

total_prompt_tokens = 0
total_completion_tokens = 0


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.1, tokens = 300)
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses_2 = list(executor.map(thread_func, range(len(personas))))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

9 {
    "Reasoning": "As Lisha Patterson, a conservative individual, I tend to disagree with President Joe Biden's policies and political views. Additionally, I consume media outlets that cater to conservative political views, which often criticize and oppose President Biden's actions. Therefore, I would disapprove of President Joe Biden's performance in office at this point in time.",
    "Answer": "Disapprove"
}
2 {
    "Reasoning": "As Sean Foster, a conservative individual who tends to vote for Republican candidates, I am likely to disapprove of President Joe Biden's performance in office. Additionally, my media consumption habits may expose me to news outlets that are critical of the current administration. However, I also believe in researching and understanding the policies and actions of elected officials before forming an opinion. Therefore, I will review President Biden's performance in office based on his policies and actions that align with my political beliefs and values."

12 {
    "Reasoning": "As Grace Davis, a left-leaning individual who prioritizes issues such as social justice, healthcare, and the environment, I approve of President Joe Biden's performance in office. He has taken steps to address these issues, such as signing executive orders to combat climate change and expand access to healthcare. Additionally, his administration has made efforts to promote diversity and inclusion, which aligns with my values. While there may be areas where I disagree with his policies, overall I believe that he is moving in the right direction and making progress on important issues.",
    "Answer": "Approve"
}
13 {
    "Reasoning": "As Felicity Ross, a conservative individual who prioritizes smaller government, lower taxes, and individual freedoms, I am likely to disapprove of President Joe Biden's performance in office. Additionally, I tend to vote for conservative candidates who align with my political beliefs and values, and President Biden's policies and act

27 {
    "Reasoning": "As Shannon Garcia, a right-leaning individual, I am likely to have some disagreements with President Joe Biden's policies and political views. However, as a responsible voter, I believe it is important to evaluate a politician's performance based on their actions and accomplishments rather than just their party affiliation. Based on my media consumption and personal research, I am aware that President Biden has taken several actions to address issues such as the COVID-19 pandemic, climate change, and social justice. While I may not agree with all of his policies, I believe that he is making an effort to address important issues and move the country forward. Therefore, I approve of President Joe Biden's performance in office at this point in time.",
    "Answer": "Approve"
}
26 {
    "Reasoning": "As Justin Martin, I am a moderate with no strong political affiliations. Therefore, I am likely to evaluate President Joe Biden's performance based on his policies and a

39 {
    "Reasoning": "As Kachina Brown, a conservative rancher from rural Montana, I tend to lean towards conservative policies and values. Therefore, I am more likely to disapprove of President Joe Biden's performance in office. Additionally, I tend to consume media outlets that align with my conservative political affiliation, which may also influence my opinion of the President's performance. However, I also believe in being an informed voter and researching each candidate and their policies before making a decision. Therefore, I will consider the actions and policies of President Biden before making a decision on whether to approve or disapprove of his performance in office.",
    "Answer": "Disapprove"
}
41 {
    "Reasoning": "As a left-leaning individual, I am more likely to approve of President Joe Biden's performance in office. Additionally, I am interested in policies that promote social justice, equality, and progressive values, which align with many of the policies that Pre

53 {
    "Reasoning": "As William Gonzales, a conservative individual who values smaller government and individual freedoms, I am likely to disapprove of President Joe Biden's performance in office. Based on my media consumption and influential figures, I am likely to consume media outlets that are critical of President Biden's policies and actions. Additionally, my voting history suggests that I tend to vote for conservative candidates who align with my political beliefs and values. Therefore, I am likely to disapprove of President Biden's performance in office due to policy disagreements and ideological differences.",
    
    "Answer": "Disapprove"
}
49 {
    "Reasoning": "As Sefu Bobcat, I tend to align myself with the Republican Party and conservative values. Therefore, I am more likely to disapprove of President Joe Biden's performance in office. Additionally, I consume media outlets that cater to conservative audiences, which may have a negative bias towards President Biden. How

63 {
    "Reasoning": "As Doris Howard, a liberal who values equal rights, access to affordable healthcare, and protecting the environment, I approve of President Joe Biden's performance in office. He has taken steps to address these issues, such as signing the American Rescue Plan Act to provide economic relief during the COVID-19 pandemic, expanding access to healthcare, and rejoining the Paris Climate Agreement. Additionally, as someone who values being an informed voter and making decisions based on facts, I appreciate President Biden's commitment to science and evidence-based policies. While there may be areas where I disagree with his policies, overall I believe he is working towards creating a more just and equitable society.",
    "Answer": "Approve"
}
66 {
    "Reasoning": "As Raymond Griffin, a left-leaning individual, I am likely to approve of President Joe Biden's performance in office. I prioritize issues such as healthcare and workers' rights, which align with Biden's pol

74 {
    "Reasoning": "As a conservative with a history of voting for the Republican Party, it is likely that Haley Coleman would disapprove of President Joe Biden's performance in office. Additionally, her media consumption habits suggest that she may consume conservative news sources that are critical of the current administration. However, it is important to note that Haley Coleman mentions that she does not blindly vote for any candidate just because they are Republican and that she always researches and evaluates their positions and track record before casting her vote. Therefore, it is possible that she may have some specific policy disagreements with the current administration but may also approve of some of their actions. Ultimately, it is difficult to say for certain without more information.",
    "Answer": "Neither Approve/Disapprove"
}
77 {
    "Reasoning": "As Violet Reed, I am a conservative and have a history of voting for the conservative party or candidate. Based on my

91 {
    "Reasoning": "As Isaac Smith, I am a conservative and have always supported the Republican Party. Based on my political affiliation, I am likely to disapprove of President Joe Biden's performance in office. Additionally, I have always believed in smaller government and lower taxes, which are not values that align with the current administration's policies. However, as a retired rancher, I am also concerned about issues affecting the agriculture industry, such as trade policies and environmental regulations. I would need to research more about the current administration's policies on these issues before making a final decision.",
    "Answer": "Disapprove"
}
90 {
    "Reasoning": "As Alexander Baker, a conservative individual who prioritizes smaller government, lower taxes, and a strong national defense, I am likely to disapprove of President Joe Biden's performance in office. President Biden's policies and actions, such as his proposed tax increases and his handling of the Afg

In [79]:
answers2 = []
for resp in responses_2:
    data = json.loads(resp)
    answers2.append(data['Answer'])
    
disapprove_count, approve_count, neither_count = count_answers(answers2)
print("Disapprove count:", disapprove_count)
print("Approve count:", approve_count)
print("Neither count:", neither_count)

Disapprove count: 52
Approve count: 36
Neither count: 12


In [80]:
responses_3 = []
#question = "Do you plan to vote in the 2024 midterm election?\n Yes\n No\n Undecided\n Please note, you are allowed to say no and not voted"

question = "Do you agree or disagree that there is a mental health crisis in Montana?\nAgree\nDisagree\nNeither Agree/Disagree"

#question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"

total_prompt_tokens = 0
total_completion_tokens = 0


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.1, tokens = 300)
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses_3 = list(executor.map(thread_func, range(len(personas))))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

3 {
    "Reasoning": "As a retired business owner living in urban Montana, I have seen the effects of mental health issues on individuals and the community. I am aware that Montana has one of the highest suicide rates in the country, and I believe that this is indicative of a larger mental health crisis. Additionally, I have seen firsthand how the lack of access to mental health resources can impact individuals and their families. Therefore, I agree that there is a mental health crisis in Montana.",
    "Answer": "Agree"
}
7 {
    "Reasoning": "As Charlotte Harris, I agree that there is a mental health crisis in Montana. While I do not have personal experience with mental health issues, I am aware that mental health resources and services are limited in rural areas like where I live. Additionally, the COVID-19 pandemic has had a significant impact on mental health, with many people experiencing increased stress, anxiety, and depression. I believe that addressing the mental health crisi

12 {
    "Reasoning": "As Grace Davis, I agree that there is a mental health crisis in Montana. While I do not have personal anecdotes related to this issue, I am aware of the high rates of suicide and substance abuse in Montana, particularly in rural areas. Additionally, I know that access to mental health services can be limited in Montana due to factors such as geographic location and lack of healthcare providers. As someone who believes in social justice and access to healthcare, I believe that addressing the mental health crisis in Montana is an important issue that requires attention and resources.",
    "Answer": "Agree"
}
17 {
    "Reasoning": "As Rosa Juarez, I am not sure if there is a mental health crisis in Montana. However, as a resident of a rural area, I know that access to mental health resources can be limited. Additionally, as someone with a lower income and education level, I know that mental health issues can be stigmatized and not always taken seriously. Therefore,

24 {
    "Reasoning": "As Koa Raccoon, I agree that there is a mental health crisis in Montana. Living in a rural area, I have seen firsthand the lack of access to mental health resources and support for individuals who are struggling with mental health issues. Additionally, as someone who has faced discrimination and marginalization, I understand the impact that systemic issues can have on mental health. Furthermore, my job as a construction worker can be physically and mentally demanding, and I have seen how the lack of support and resources can take a toll on individuals' mental health. Overall, based on my personal experiences and observations, I believe that there is a mental health crisis in Montana.",
    "Answer": "Agree"
}
25 {
    "Reasoning": "As Kristine Scott, I agree that there is a mental health crisis in Montana. In my role as a Veteran Outreach Coordinator, I have seen firsthand the impact that mental health issues can have on individuals and communities. Montana has a

43 {
    "Reasoning": "As Louis Kelly, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware of the high rates of suicide and substance abuse in Montana, particularly in rural areas like where I live. I also know that access to mental health services can be limited in rural areas, which can exacerbate the problem. Additionally, I believe that the COVID-19 pandemic has only made the mental health crisis worse, as people are dealing with increased stress, isolation, and uncertainty.",
    "Answer": "Agree"
}
42 {
    "Reasoning": "As Samuel Romero, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware of the impact that mental health can have on individuals and communities. As a construction worker, I have seen firsthand the toll that stress and anxiety can take on people in high-pressure jobs. Additionally, I am aware of the 

52 {
    "Reasoning": "As an 18-year-old retail sales associate from an urban area in Montana, I may not have a comprehensive understanding of the mental health situation in the state. However, based on my personal experiences and what I have heard from others, I believe that there is a mental health crisis in Montana. I have seen friends and family members struggle with mental health issues, and I know that access to mental health resources can be limited in rural areas. Additionally, I have heard about the high suicide rates in Montana, which is a clear indication that there are significant mental health challenges in the state. Therefore, I agree that there is a mental health crisis in Montana.",
    "Answer": "Agree"
}
54 {
    "Reasoning": "As Edith Brown, I agree that there is a mental health crisis in Montana. While I do not have any personal anecdotes related to mental health, I am aware of the challenges that individuals in my community face when it comes to accessing mental h

68 {
    "Reasoning": "As Jamie Ward, I agree that there is a mental health crisis in Montana. Although I do not have any specific personal anecdotes related to mental health, I am aware of the challenges that many individuals face in accessing mental health services, particularly in rural areas like Montana. Additionally, as a retired nurse, I understand the importance of mental health and the impact that it can have on overall well-being. Based on my media consumption and political affiliation, I am also likely to be aware of the current discussions and debates surrounding mental health policy and funding. Therefore, I believe that there is a mental health crisis in Montana and that more needs to be done to address this issue.",
    "Answer": "Agree"
}
71 {
    "Reasoning": "As Preston Taylor, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware that access to mental healthcare in rural areas can be li

86 {
    "Reasoning": "As Colin Lopez, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware that Montana has a high suicide rate and limited access to mental health resources. Additionally, the COVID-19 pandemic has likely exacerbated mental health issues for many people. I believe that it is important for our community and political leaders to prioritize mental health and work towards increasing access to resources and reducing the stigma surrounding mental health issues.",
    "Answer": "Agree"
}
88 {
    "Reasoning": "As a retired farmer living in rural Montana, I have seen firsthand the challenges that rural residents face in accessing quality healthcare. Mental health services are often limited in rural areas, and many people may not have access to the care they need. Additionally, the isolation and lack of social support that can come with living in a rural area may contribute to mental health issu

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 04f94693a3d78cc34840f4c42c046c66 in your message.)
85 {
    "Reasoning": "As Madison Perez, I agree that there is a mental health crisis in Montana. As a retired veteran, I have seen firsthand the impact that mental health issues can have on individuals and their families. Additionally, as a member of a local senior citizens group, I have heard stories from others in my community about the challenges they have faced in accessing mental health services. Based on my personal experiences and the information available, I believe that there is a significant need for increased resources and support for mental health in Montana.",
    "Answer": "Agree"
}
Total Prompt Tokens: 118151
Total Completion Tokens: 12195

Token Cost = $ 0.26069200000000003


In [14]:
def count_answers_agree(strings):
    count_disapprove = 0
    count_approve = 0
    count_neither = 0

    for string in strings:
        if string == "Disagree":
            count_disapprove += 1
        elif string == "Agree":
            count_approve += 1
        elif string == "Neither Agree/Disagree":
            count_neither += 1

    return count_disapprove, count_approve, count_neither

In [83]:
answers3 = []
for resp in responses_3:
    data = json.loads(resp)
    answers3.append(data['Answer'])
    
disapprove_count, approve_count, neither_count = count_answers_agree(answers3)
print("Disapprove count:", disapprove_count)
print("Approve count:", approve_count)
print("Neither count:", neither_count)

Disapprove count: 0
Approve count: 100
Neither count: 0


In [94]:
def person_to_string(person):
    persona_string = []
    for key, value in person.items():
        # Some keys require specific phrasing to form a directive sentence
        if key == 'Name':
            persona_string.append(f"Your name is {value}.")
        elif key == 'Age':
            persona_string.append(f"You are {value} years old.")
        elif key == 'Ethnicity':
            persona_string.append(f"You identify as {value}.")
        elif key == 'Gender':
            persona_string.append(f"You are a {value} individual.")
        elif key == 'Income':
            persona_string.append(f"You make an annual income of around {value} USD.")
        elif key == 'Education Level':
            persona_string.append(f"Your highest level of education is {value}.")
        elif key == 'Political Affiliation':
            persona_string.append(f"You lean towards the {value} politically.")
        elif key == 'Geographic Location':
            persona_string.append(f"You currently live in {value}.")
        elif key == 'Veteran Status':
            persona_string.append(f"You are {value}.")
        elif key == 'Occupation':
            persona_string.append(f"You work as a {value}.")
        else:
            # The remaining keys don't require specific phrasing, so we can just add their values
            # These are written in first person so we can use them directly
            persona_string.append(value)
    
    # Join all elements of persona_string into a single string, separated by space
    return " ".join(persona_string)

In [24]:
responses_4 = []
#question = "Do you plan to vote in the 2024 midterm election?\n Yes\n No\n Undecided\n Please note, you are allowed to say no and not voted"

question = "Do you agree or disagree with the Supreme Court's decision to overturn Roe v. Wade?\nAgree\nDisagree\nNeither Agree/Disagree"

#question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"

total_prompt_tokens = 0
total_completion_tokens = 0


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-4' , role = persona_role, temp = 0.5, tokens = 600)
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses_4 = list(executor.map(thread_func, range(10)))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

5 {
  "Reasoning": "As Mae Foster, a 35-year-old conservative woman living in rural Montana, I have been raised with traditional values and a belief in individual freedoms. My conservative political affiliation and admiration for figures like Ronald Reagan have shaped my views on various issues, including abortion. I am concerned about the potential for government overreach and the erosion of individual freedoms.",
  "Answer": "Agree"
}
9 {
  "Reasoning": "As Lisha Patterson, I am a conservative and I believe in limited government and individual responsibility. I am also a Black woman who has experienced discrimination and inequality, so I understand the importance of equal rights and opportunities. When it comes to the Supreme Court's decision to overturn Roe v. Wade, I have to consider my conservative values and my personal experiences.",
  "Answer": "Agree"
}
8 {
  "Reasoning": "As a conservative, I tend to lean towards limited government intervention and upholding the Constitution.

In [26]:
answers4 = []
for resp in responses_4:
    try:
        data = json.loads(resp)
        answers4.append(data['Answer'])
    except:
        continue
    
disapprove_count, approve_count, neither_count = count_answers_agree(answers4)
print("Disapprove count:", disapprove_count)
print("Approve count:", approve_count)
print("Neither count:", neither_count)

Disapprove count: 0
Approve count: 9
Neither count: 1
